# Inspect GloVe

### @edufierro
----

In [1]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

data_dir = "/Users/eduardofierro/Google Drive/TercerSemetre/Optimization/Project/Data/"

In [2]:
glove = pickle.load( open(data_dir +  "GloVe_at_1900.p", "rb" ) )
index_to_word_map = pickle.load( open(data_dir +  "index_to_word_map.p", "rb" ) )
word_to_index_map = pickle.load( open(data_dir +  "word_to_index_map.p", "rb" ) )

In [3]:
normalized_glove = np.copy(glove)
for col in range(0, normalized_glove.shape[0]):
    normalized_glove[col, :] = normalized_glove[col, :] / np.linalg.norm(normalized_glove[col, :])

# Similarity

In [69]:
def similarity(matrix, word_one, word_two):
    
    vec_one = matrix[word_to_index_map[word_one]].reshape(1, -1)
    vec_two = matrix[word_to_index_map[word_two]].reshape(1, -1)
    return float(cosine_similarity(vec_one, vec_two))

def nearest(matrix, word): 
    
    nearest_val = similarity(matrix, word, index_to_word_map[0])
    nearest_index = 0
    
    for x in range(1, matrix.shape[0]):
        score = similarity(matrix, word, index_to_word_map[x])
        if score < nearest_val: 
            nearest_val = score
            nearest_index = x
    
    return nearest_index

def nearest_from_vector(matrix, vec, k = 5): 
    
    vec = vec.reshape(1, -1)
    all_scores = []
    
    for x in range(0, matrix.shape[0]):
        all_scores.append((x,float(cosine_similarity(vec, matrix[x].reshape(1, -1)))))

    all_scores = sorted(all_scores, key=lambda key: key[1])    
    all_scores = all_scores[0:k]
    return [all_scores[x][0] for x in range(0, k)]

def word_analogy(matrix, word_a, word_b, word_c, k=5):
    """
    Function that solves problem word_a to word_b = word_c to ?
    Remember to use normalized_embeddings, why?
    @param word_a, word_b, word_c: string
    @param k: top k candidates to return
    """
    
    vec = matrix[word_to_index_map[word_b]] - matrix[word_to_index_map[word_a]] + matrix[word_to_index_map[word_c]]
    nearest = nearest_from_vector(matrix, vec, k = k)
    words = []
    for x in range(0, k):
        words.append(index_to_word_map[nearest[x]])
    
    return words